<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Pipeline de Dados do Telegram II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Ingestão;</li>
  <li>ETL;</li>
  <li>Apresentação;</li>
  <li>Storytelling.</li>
</ol>

---

# **Exercícios**

## 1\. Ingestão

Replique as atividades da aula 1.

In [15]:
# injetar a API através do token no python

from getpass import getpass

token = getpass()

··········


In [16]:
# criar URL base

import json

import requests

base_url = f'https://api.telegram.org/bot{token}'

In [17]:
url = f"https://api.telegram.org/bot{token}/deleteWebhook"

r = requests.get(url)
print(r.json())

{'ok': True, 'result': True, 'description': 'Webhook is already deleted'}


- getME

In [18]:
# retorna informações do BOT

response = requests.get(url=f'{base_url}/getMe')
print(f'{base_url}/getMe')

print(json.dumps(json.loads(response.text), indent=2))

https://api.telegram.org/bot8346440950:AAHLHI74Uj2vsCl6QpP7ky9kX3zg7AD5Mf0/getMe
{
  "ok": true,
  "result": {
    "id": 8346440950,
    "is_bot": true,
    "first_name": "data_pipeline_ebac_bot",
    "username": "data_pipeline_ebac_bot",
    "can_join_groups": false,
    "can_read_all_group_messages": false,
    "supports_inline_queries": false,
    "can_connect_to_business": false,
    "has_main_web_app": false
  }
}


- getUpdates

In [19]:
# retorna as mensagens captadas pelo BOT
response = requests.get(url=f'{base_url}/getUpdates')

print(json.dumps(json.loads(response.text), indent=2))

{
  "ok": true,
  "result": [
    {
      "update_id": 144169603,
      "message": {
        "message_id": 20,
        "from": {
          "id": 8518432228,
          "is_bot": false,
          "first_name": "Nathalia",
          "last_name": "Sa",
          "language_code": "pt-br"
        },
        "chat": {
          "id": -1003415167553,
          "title": "Data Pipeline EBAC",
          "type": "supergroup"
        },
        "date": 1763674352,
        "text": "mensagem para enriqucer"
      }
    },
    {
      "update_id": 144169604,
      "message": {
        "message_id": 21,
        "from": {
          "id": 8518432228,
          "is_bot": false,
          "first_name": "Nathalia",
          "last_name": "Sa",
          "language_code": "pt-br"
        },
        "chat": {
          "id": -1003415167553,
          "title": "Data Pipeline EBAC",
          "type": "supergroup"
        },
        "date": 1763674358,
        "text": "base de dados"
      }
    },
    {
      "u

1.1. Crie um `bucket` no `AWS S3` para o armazenamento de dados crus, não se esqueça de adicionar o sufixo `-raw`.

- bucket criado no AWS S3 = project-ebac-datalake-raw

1.2. Crie uma função no `AWS Lambda` para recebimento das mensagens e armazenamento no formato JSON no `bucket` de dados crus. Não se esqueça de configurar as variáveis de ambiente e de adicionar as permissão de interação com `AWS S3` no `AWS IAM`.

In [20]:
!python --version

Python 3.12.12


> **Nota**: Para testar a função com evento do próprio `AWS Lambda`, substitua o código `message = json.loads(event["body"])` por `message = event`. Lembre-se que o primeiro só faz sentido na integração com o `AWS API Gateway`.

1.3. Crie uma API no `AWS API Gateway` a conecte a função do `AWS Lambda`, conforme apresentado na aula.

> **Nota**: não disponibilize o endereço da API gerada.

In [9]:
aws_api_gateway_url = getpass()

··········


1.4. Configura o *webhook* do *bot* através do método `setWebhook` da API de *bots* do **Telegram**. utilize o endereço da API criada no `AWS API Gateway`. Utilize o método `getWebhookInfo` para consultar a integração.

> **Nota**: não disponibilize o *token* de acesso ao seu *bot* da API de *bots* do **Telegram**.

- setWebhook

In [21]:
response = requests.get(url=f'{base_url}/setWebhook?url={aws_api_gateway_url}')

print(json.dumps(json.loads(response.text), indent=2))

{
  "ok": true,
  "result": true,
  "description": "Webhook was set"
}


 - **getWebhookInfo**

In [22]:
response = requests.get(url=f'{base_url}/getWebhookInfo')

print(json.dumps(json.loads(response.text), indent=2))

{
  "ok": true,
  "result": {
    "url": "https://6kyhne9bs4.execute-api.sa-east-1.amazonaws.com/dev",
    "has_custom_certificate": false,
    "pending_update_count": 2,
    "max_connections": 40,
    "ip_address": "54.233.87.71"
  }
}


## 2\. ETL

Replique as atividades da aula 2.

2.1. Crie um `bucket` no `AWS S3` para o armazenamento de dados enriquecidos, não se esqueça de adicionar o sufixo `-enriched`.

2.2. Cria uma função no `AWS Lambda` para processar as mensagens JSON de uma única partição do dia anterior (D-1), armazenadas no *bucket* de dados crus. Salve o resultado em um único arquivo PARQUET, também particionado por dia. Não se esqueça de configurar as variáveis de ambiente, de adicionar as permissão de interação com `AWS S3` no `AWS IAM`, de configurar o *timeout* e de adicionar a *layer* com o código do pacote Python PyArrow.

> **Nota**: Para testar a função, substitua o código `date = (datetime.now(tzinfo) - timedelta(days=1)).strftime('%Y-%m-%d')` por `date = (datetime.now(tzinfo) - timedelta(days=0)).strftime('%Y-%m-%d')`, permitindo assim o processamento de mensagens de um mesmo dia.

2.3. Crie uma regra no `AWS Event Bridge` para executar a função do `AWS Lambda` todo dia a meia noite no horário de Brasília (GMT-3).

## 3\. Apresentação

Replique as atividades da aula 3.

3.1. Cria a tabela no `AWS Athena` que aponte para os dados armazenados no bucket enriquecido do `AWS S3`.

```sql
CREATE EXTERNAL TABLE `telegram`(
  `message_id` bigint,
  `user_id` bigint,
  `user_is_bot` boolean,
  `user_first_name` string,
  `chat_id` bigint,
  `chat_type` string,
  `text` string,
  `date` bigint)
PARTITIONED BY (
  `context_date` date)
ROW FORMAT SERDE
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION
  's3://project-ebac-datalake-enriched/telegram/'
```

3.2. Execute o comando `MSCK REPAIR TABLE <nome-tabela>` para carregar as partições.

```sql
MSCK REPAIR TABLE `telegram`;
```

3.3. Execute as consultas SQL para explorar os dados.

```sql
SELECT * FROM "telegram"
ORDER BY message_id
LIMIT 10;
```

3.4. Analytics

Obs. No momento de consulta foi no mesmo dia de implantação do bot, então temos apenas 1 dia de análise e de mensagens testes de um unico usuário.

- Média do tamanho das mensagens por usuário por dia.

```sql
SELECT
  user_id,
  user_first_name,
  context_date,
  CAST(AVG(length(text)) AS INT) AS "average_message_length"
FROM "telegram"
GROUP BY
  user_id,
  user_first_name,
  context_date
ORDER BY context_date DESC
```

average_message_length = 24

- Quantidade de mensagens por dia.

```sql
SELECT
  context_date,
  count(1) AS "message_amount"
FROM "telegram"
GROUP BY context_date
ORDER BY context_date DESC
```

message_amount = 9

## 4\. Storytelling

4.1 Contexto:

Chatbots como os do Telegram geram uma grande quantidade de dados, registrando cada interação dos usuários em tempo real.

Para transformar essas informações em insights, foi criada uma pipeline completa na AWS que converte dados transacionais em dados analíticos, já tratados e prontos para exploração no Athena.

Este projeto mostra exatamente como essa jornada acontece: Do Bot criado, da mensagem enviada no grupo Telegram, da captação e ingestão das mensagens até uma base estruturada para análise.



![Fluxograma Pipeline de Dados](https://raw.githubusercontent.com/nathalia-de-sa/Telegram-Data-Pipeline-/refs/heads/main/Fluxo%20de%20Processamento%20de%20Dados.png)

4.2 Estrutura Arquitetura:

- Sistema transacional:
  - Bot criado no Telegram, apenas para a leitura das mensagens.
  - Utilizando a API fornecida pelo Telegram para possibilitar as conexões
  - Uso do método getUpdates para capturar mensagens do grupo no notebook Python, e poder transformá-las em arquivo JSON.

- Sistema analítico (AWS):

  - Ingestão: API Gateway + Lambda recebem os dados e armazenam no S3.

  - ETL: uma segunda função Lambda consome esses dados brutos, transforma e salva a nova versão processada no S3.

  - Criação de automatização: Utilizando o Event Bridge da AWS para dar sequencia na ingestão dos dados diariamente de forma automatizada e armazenar no S3 de forma particionada

  - Apresentação: uso do AWS Athena para consulta SQL dos dados já modelados.

4.3 Análise Exploratória de Dados:

Dicionário de dados:

| Campo               | Tipo    | Descrição                                                                                      |
| ------------------- | ------- | ---------------------------------------------------------------------------------------------- |
| **message_id**      | bigint  | Identificador único da mensagem dentro do chat. Permite rastrear e diferenciar cada envio.     |
| **user_id**         | bigint  | ID único do usuário que enviou a mensagem. Serve para identificar o autor e analisar padrões.  |
| **user_is_bot**     | boolean | Indica se o autor da mensagem é um bot (`true`) ou um usuário humano (`false`).                |
| **user_first_name** | string  | Primeiro nome do usuário que enviou a mensagem. Útil para análises descritivas e agrupamentos. |
| **chat_id**         | bigint  | Identificador único do chat ou grupo no Telegram. Permite diferenciar conversas ou canais.     |
| **chat_type**       | string  | Tipo de chat onde a mensagem foi enviada (ex.: `private`, `group`, `supergroup`).              |
| **text**            | string  | Conteúdo textual da mensagem. Pode incluir comandos, perguntas e interações do usuário.        |
| **date**            | bigint  | Timestamp Unix da mensagem (segundos desde 01/01/1970). Usado para análises temporais.         |


4.4 Melhorias:

Melhoria mapeada para implantação em roadmap:
  - Ingestão de outros formatos além do texto, como imagens e aúdios compartilhados no grupo.
